In [1]:
import numpy as np
import pandas as pd
from scipy.stats import f_oneway
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [2]:
# Example Data (replace with your actual data)
np.random.seed(0)
trap1_data = np.random.rand(4, 10)
trap2_data = np.random.rand(4, 10)

# Combine into a single DataFrame with labels for ANOVA
trap1_df = pd.DataFrame(trap1_data, columns=[f'Element_{i+1}' for i in range(10)])
trap2_df = pd.DataFrame(trap2_data, columns=[f'Element_{i+1}' for i in range(10)])
trap1_df['TrapType'] = 'Trap1'
trap2_df['TrapType'] = 'Trap2'

combined_df = pd.concat([trap1_df, trap2_df], ignore_index=True)

In [3]:
def clr_manual(data):
    # Compute geometric mean
    geometric_mean = np.exp(np.mean(np.log(data + 1e-10), axis=1))  # Add small value to avoid log(0)
    clr_transformed = np.log((data.T + 1e-10) / geometric_mean).T
    return clr_transformed

# Apply CLR transformation to the data (excluding the 'TrapType' column)
clr_transformed_data = clr_manual(combined_df.drop('TrapType', axis=1))
# Convert CLR data to DataFrame and add the TrapType column back
clr_df = pd.DataFrame(clr_transformed_data)
clr_df['TrapType'] = combined_df['TrapType']

In [8]:
# Conduct ANOVA on each element's CLR transformed data
anova_results = {}
for column in clr_df.columns[:-1]:  # Exclude 'TrapType' column
    model = ols(f'{column} ~ C(TrapType)', data=clr_df).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)
    anova_results[column] = anova_table

# Display ANOVA results for each element
for element, result in anova_results.items():
    print(f"ANOVA results for {element}:")
    print(result, "\n")

# Overall ANOVA using the mean clr scores across elements
clr_df['mean_clr'] = clr_df.drop('TrapType', axis=1).mean(axis=1)
model_mean_clr = ols('mean_clr ~ C(TrapType)', data=clr_df).fit()
anova_mean_clr = sm.stats.anova_lm(model_mean_clr, typ=2)
print("ANOVA results for mean CLR scores:")
print(anova_mean_clr)


ANOVA results for Element_1:
               sum_sq   df         F    PR(>F)
C(TrapType)  0.012544  1.0  0.030115  0.867935
Residual     2.499094  6.0       NaN       NaN 

ANOVA results for Element_2:
               sum_sq   df         F    PR(>F)
C(TrapType)  0.312781  1.0  1.761558  0.232698
Residual     1.065355  6.0       NaN       NaN 

ANOVA results for Element_3:
               sum_sq   df          F    PR(>F)
C(TrapType)  1.439270  1.0  31.539166  0.001361
Residual     0.273806  6.0        NaN       NaN 

ANOVA results for Element_4:
               sum_sq   df         F    PR(>F)
C(TrapType)  2.528133  1.0  4.191052  0.086576
Residual     3.619329  6.0       NaN       NaN 

ANOVA results for Element_5:
               sum_sq   df         F    PR(>F)
C(TrapType)  6.646790  1.0  7.624393  0.032801
Residual     5.230678  6.0       NaN       NaN 

ANOVA results for Element_6:
               sum_sq   df         F    PR(>F)
C(TrapType)  0.144647  1.0  0.125932  0.734828
Residual     6